In [1]:
import pandas as pd
import cv2
import os

In [2]:
mos_df = pd.read_csv("../data/MOS.csv") 
print(mos_df.head())

       image_id     mean      std
0  I01_01_1.bmp  5.51429  0.13013
1  i01_01_2.bmp  5.56757  0.16008
2  i01_01_3.bmp  4.94444  0.15186
3  i01_01_4.bmp  4.37838  0.11046
4  i01_01_5.bmp  3.86486  0.10229


---
#### imread stores image as a 3D array [height, width, channels]
#### cvtColor converts the color space from BGR to RBG 
---

In [3]:
img_path = os.path.join("../data/distorted_images", mos_df['image_id'][0])
img = cv2.imread(img_path)
print(img_path)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

../data/distorted_images\I01_01_1.bmp


---

#### Takes in an image as a parameter 
#### Converts it to grayscale 
#### Laplacian takes the second derivative of pixel intensities and then we calculate variance to decide if the image is sharp 
#### Brightness is determined by the average pixel intensity of the grayscale image 
#### Noise is determined by the average standard deviation of the grayscale image 
#### color_var = np.std(img, axis=(0,1)).mean() calculates the standard deviation across each color channel and then takes the mean to see if it is monochrome or saturated 

In [4]:
import numpy as np
def extract_features(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    # Sharpness
    sharpness = cv2.Laplacian(gray, cv2.CV_64F).var()

    # Brightness & noise
    brightness = np.mean(gray)
    noise = np.std(gray)

    # Color channel stats
    mean_b, mean_g, mean_r = np.mean(img, axis=(0,1))
    std_b, std_g, std_r = np.std(img, axis=(0,1))

    # Entropy
    hist, _ = np.histogram(gray, bins=256, range=(0,256), density=True)
    entropy_val = -np.sum(hist * np.log2(hist + 1e-7))

    # Edge density
    edges = cv2.Canny(gray, 100, 200)
    edge_density = np.sum(edges > 0) / edges.size

    return [
        sharpness, brightness, noise,
        mean_b, mean_g, mean_r,
        std_b, std_g, std_r,
        entropy_val, edge_density
    ]

--- 
#### Features store all the feature data and labels store the mean opinion score for the image 
#### This function loads each image in the distorted data folder, converts it to RGB, extracts its features, and appends the mean optimised score for the image to the row

In [5]:
features = []
labels = []

for idx, row in mos_df.iterrows():
    img_path = os.path.join("../data/distorted_images", row['image_id'])
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    features.append(extract_features(img))
    labels.append(row['mean'])

import pandas as pd

columns = [
    'sharpness', 'brightness', 'noise',
    'mean_b', 'mean_g', 'mean_r',
    'std_b', 'std_g', 'std_r',
    'entropy', 'edge_density'
]

features_df = pd.DataFrame(features, columns=columns)
features_df['mean'] = labels

In [6]:
features_df.describe()

,sharpness,brightness,noise,mean_b,mean_g,mean_r,std_b,std_g,std_r,entropy,edge_density,mean
count,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000,3000.000000
mean,3090.827752,110.978255,48.926543,120.747306,109.754216,91.615236,51.206428,51.726293,48.564487,6.918131,0.118361,4.474657
std,5174.869286,25.223155,12.731646,22.744159,28.256033,28.458487,12.967691,12.894053,13.828087,1.048619,0.071929,1.239880
min,1.146698,37.158381,12.874847,48.850764,18.184525,5.022507,14.184254,13.588187,12.216761,0.308947,0.000000,0.242420
25%,673.165757,91.945970,40.082295,108.005183,82.576115,69.625641,43.615132,43.010903,39.476830,6.998911,0.055908,3.611110
50%,1578.196417,107.922424,47.560431,120.549662,108.881930,87.319036,50.179852,50.163549,47.403506,7.221295,0.111468,4.600000
75%,3014.812871,123.465853,53.874086,134.411105,124.327924,112.456511,59.102299,57.315007,53.302752,7.453756,0.171759,5.526320
max,39899.549265,192.376226,103.901088,196.654032,195.159999,178.966777,102.975576,104.284358,110.616082,7.897516,0.342311,7.214290


In [7]:
from sklearn.ensemble import GradientBoostingRegressor

gbm = GradientBoostingRegressor(random_state=42) 

gbm_param_dist = {
    'n_estimators': [400, 600, 800, 1000],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 4, 5, 6],
    'min_samples_split': [5, 10, 15],
    'min_samples_leaf': [2, 4, 6],
    'subsample': [0.7, 0.8, 0.9, 1.0],
    'max_features': ['sqrt', 'log2', None]
}

In [8]:
from sklearn.model_selection import RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=gbm,
    param_distributions=gbm_param_dist,
    n_iter=50,                # 50 random combinations
    cv=5,                     # 5-fold cross-validation
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=2,
    random_state=42
)

In [9]:
gbm_params = {
    "loss": "squared_error",
    "learning_rate": 0.05,
    "n_estimators": 800,
    "subsample": 0.7, 
    "criterion": "friedman_mse",
    "min_samples_split": 10,
    "min_samples_leaf": 5,
    "min_weight_fraction_leaf": 0.0,
    "max_depth": 5,
    "min_impurity_decrease": 0.0,
    "init": None,
    "random_state": 42,
    "max_features": "log2",
    "alpha": 0.9,
    "verbose": 0,
    "max_leaf_nodes": None,
    "warm_start": False,
    "validation_fraction": 0.1,
    "n_iter_no_change": None,
    "tol": 0.0001,
    "ccp_alpha": 0.0
}


In [10]:
from sklearn.model_selection import train_test_split

X = features_df.drop(columns=['mean'])
y = features_df['mean']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [11]:
best_gbm = GradientBoostingRegressor(**gbm_params) 

best_gbm.fit(X_train, y_train)

,loss,'squared_error'
,learning_rate,0.05
,n_estimators,800
,subsample,0.7
,criterion,'friedman_mse'
,min_samples_split,10
,min_samples_leaf,5
,min_weight_fraction_leaf,0.0
,max_depth,5
,min_impurity_decrease,0.0
,init,None


In [12]:
y_pred = best_gbm.predict(X_test)

from sklearn.metrics import mean_squared_error, mean_absolute_error
print("Test MSE:", mean_squared_error(y_test, y_pred))
print("Test MAE:", mean_absolute_error(y_test, y_pred)) 

Test MSE: 0.580685189504513
Test MAE: 0.5900851346203329


In [13]:
print("Train MSE:", mean_squared_error(y_train, best_gbm.predict(X_train)))
print("Test MSE:", mean_squared_error(y_test, best_gbm.predict(X_test))) 

Train MSE: 0.07226938066910504
Test MSE: 0.580685189504513
